In [ ]:
! pip install -q transformers==4.52.2
! pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 56.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
from torch import nn
from transformers.models.opt.modeling_opt import OPTAttention

import sys
sys.path.append('/content/drive/MyDrive/Transformers/Project_OPT/Grouping')

! pip install -q k_means_constrained
from Grouping_Utils import close_by_w, close_by_attention_score, plot_layer_head_similarity

# **Load Model**

In [ ]:
n_heads = 32
kv_heads = 32

repo_name = "facebook/opt-1.3b"
model = AutoModelForSequenceClassification.from_pretrained(
   repo_name, num_labels=3, device_map="cpu",
)
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-1.3b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


OPTForSequenceClassification(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer

In [ ]:
print(model.get_memory_footprint()/1e6)
model

5263.056896


OPTForSequenceClassification(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 2048, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 2048)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-23): 24 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
          (final_layer

# **Plot heads per layer similarity**
By generating random input x, the similarity score will be obtained from the Wk/Wq/Wv times x output.

In [ ]:
layer_idx = 0
total_grouping = []

for name, module in model.model.decoder.named_modules():
  if isinstance(module, OPTAttention):
    kvq_w = module.q_proj.weight.data
    layer_grouping, _ = close_by_w(kvq_w, n_heads, kv_heads, layer_idx, group_size=2)
    total_grouping.append(layer_grouping)
    layer_idx += 1

In [ ]:
l = [arr.astype(int).tolist() for arr in total_grouping]
print(l)

# **By sampling input from the validation set of MNLI**

In [ ]:
from datasets import load_dataset

dataset = load_dataset("nyu-mll/glue", "mnli")

tokenizer = AutoTokenizer.from_pretrained(repo_name)
tokenizer.pad_token = tokenizer.eos_token

dataset

DatasetDict({
    train: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 392702
    })
    validation_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9815
    })
    validation_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9832
    })
    test_matched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['premise', 'hypothesis', 'label', 'idx'],
        num_rows: 9847
    })
})

In [ ]:
dataset["train"] = dataset["train"].shuffle(seed=42).select(range(50000))

In [ ]:
def preprocess(example):
    return tokenizer(
        example["premise"],
        example["hypothesis"],
        truncation=True,
        padding="max_length",
        max_length=128,
    )

encoded_dataset = dataset.map(preprocess, batched=True)
encoded_dataset = encoded_dataset.rename_column("label", "labels")
encoded_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

In [ ]:
import torch
import torch.nn.functional as F
from transformers.models.opt.modeling_opt import OPTAttention

def capture_projection2(model, input_ids, attention_mask, n_heads, proj_type="q"):
    captured = []

    def save_output(module, input, output):
        # output is the projected tensor (batch, seq_len, hidden_dim)
        captured.append(output.detach())

    hooks = []
    for name, module in model.model.decoder.named_modules():
        if isinstance(module, torch.nn.Linear) and name.endswith(f"{proj_type}_proj"):
            hooks.append(module.register_forward_hook(save_output))

    with torch.no_grad():
        model(input_ids=input_ids, attention_mask=attention_mask)

    for h in hooks:
        h.remove()

    # Reshape for heads
    for i in range(len(captured)):
        d = captured[i].shape[-1] // n_heads
        captured[i] = captured[i].view(captured[i].shape[0], -1, n_heads, d).transpose(1, 2)

    print(f"Captured {proj_type}-projection from {len(captured)} layers.")
    print("Shape of first capture:", captured[0].shape)
    return captured

# Example use
sample = encoded_dataset["validation_matched"][0]
input_ids = sample["input_ids"].unsqueeze(0).clone().detach()
attention_mask = sample["attention_mask"].unsqueeze(0).clone().detach()

final_out = []

for idx in range(10):
    sample = encoded_dataset["validation_matched"][idx]
    input_ids = sample["input_ids"].unsqueeze(0).clone().detach()
    attention_mask = sample["attention_mask"].unsqueeze(0).clone().detach()

    q_out = capture_projection2(model, input_ids, attention_mask, n_heads, proj_type="q")

    if not final_out:
        final_out = q_out
    else:
        for j in range(len(q_out)):
            final_out[j] = torch.cat((final_out[j], q_out[j]), dim=2)

print("Final shape of first layer's k projection:", final_out[0].shape)

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns

def plot_layer_head_similarity(outs):
    for layer_idx, k_proj in enumerate(outs):
        k_proj = k_proj.squeeze(0)  # (n_heads, seq_len, d_kv)
        n_heads, seq_len, d_kv = k_proj.shape

        sim_matrix = torch.zeros((n_heads, n_heads))

        for t in range(seq_len):
            k_vectors = k_proj[:, t, :]  # (n_heads, d_kv)
            k_norm = F.normalize(k_vectors, dim=1)  # normalize each head's vector

            cos_sim = torch.matmul(k_norm, k_norm.T)  # → (n_heads, n_heads)
            sim_matrix += cos_sim

        # Average across sequence length
        sim_matrix /= seq_len
        sim_matrix_np = sim_matrix.cpu().numpy()

        plt.figure(figsize=(6, 5))
        sns.heatmap(sim_matrix_np, cmap='coolwarm', vmin=-1, vmax=1,
                    xticklabels=[f"H{i}" for i in range(n_heads)],
                    yticklabels=[f"H{i}" for i in range(n_heads)])
        plt.title(f"Layer {layer_idx} - Head-to-Head K Cosine Similarity")
        plt.xlabel("Head")
        plt.ylabel("Head")
        plt.tight_layout()
        plt.show()

plot_layer_head_similarity(q_out)

In [ ]:
from Grouping_Utils import close_wk, plot_layer_head_similarity, process_grouping

total_grouping = []

for layer in range(len(q_out)):
  grouping = plot_layer_head_similarity(q_out[layer], layer, 2)
  grouping = process_grouping(grouping, n_heads)

  total_grouping.append(grouping)


l = [arr.astype(int).tolist() for arr in total_grouping]
print(l)